In [204]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline

In [123]:
df = pd.read_csv('outbreaks.csv')

In [125]:
df = df[df['Food'].notnull()][df['Species'].notnull()]

/home/jackdougherty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [140]:
df.head()

,Year,Month,State,Location,Food,Ingredient,Species,Serotype/Genotype,Status,Illnesses,Hospitalizations,Fatalities
3,1998,January,California,Restaurant,"Fish, Ahi",NaN,Scombroid toxin,NaN,Confirmed,4,0.0,0.0
4,1998,January,California,Private Home/Residence,"Lasagna, Unspecified; Eggs, Other",NaN,Salmonella enterica,Enteritidis,Confirmed,26,3.0,0.0
7,1998,January,California,Restaurant,"Stuffing, Unspecified; Sandwich, Turkey",NaN,Salmonella enterica,Enteritidis,Confirmed,4,3.0,0.0
8,1998,January,Colorado,Restaurant,"Salad, Unspecified",NaN,Virus,NaN,Suspected,21,NaN,NaN
9,1998,January,Colorado,Restaurant,"Peppers, Chili",NaN,Virus,NaN,Suspected,3,NaN,0.0


In [145]:
df['Food'] = df['Food'].str.lower()

In [147]:
df['Species'] = df['Species'].str.lower()

In [187]:
df.head(20)

,Year,Month,State,Location,Food,Ingredient,Species,Serotype/Genotype,Status,Illnesses,Hospitalizations,Fatalities
3,1998,January,California,Restaurant,"fish, ahi",NaN,scombroid toxin,NaN,Confirmed,4,0.0,0.0
4,1998,January,California,Private Home/Residence,"lasagna, unspecified; eggs, other",NaN,salmonella enterica,Enteritidis,Confirmed,26,3.0,0.0
7,1998,January,California,Restaurant,"stuffing, unspecified; sandwich, turkey",NaN,salmonella enterica,Enteritidis,Confirmed,4,3.0,0.0
8,1998,January,Colorado,Restaurant,"salad, unspecified",NaN,virus,NaN,Suspected,21,NaN,NaN
9,1998,January,Colorado,Restaurant,"peppers, chili",NaN,virus,NaN,Suspected,3,NaN,0.0
15,1998,January,Florida,Restaurant,"ethnic style, unspecified",NaN,clostridium perfringens,NaN,Suspected,3,0.0,0.0
16,1998,January,Florida,Restaurant,greek salad,NaN,norovirus genogroup i,NaN,Suspected,2,NaN,NaN
17,1998,January,Florida,Restaurant,"ground beef, cheeseburger",NaN,staphylococcus aureus,NaN,Suspected,2,0.0,0.0
18,1998,January,Florida,Restaurant,"pasta, unspecified",NaN,bacillus cereus,NaN,Suspected,2,0.0,0.0
19,1998,January,Florida,Restaurant,"sauces, unspecified; tuna, unspecified",NaN,scombroid toxin,NaN,Suspected,2,2.0,0.0


In [159]:
species = df['Species'].str.lower().str.replace(';', '').str.split(expand=True)

In [166]:
toxins = []
for row in range(species.shape[0]):
    toxins.extend(species.iloc[row,:].unique())
toxins = list(set([t.strip() for t in toxins if t is not None]))
stop_list = ['(msg)', 'herbal', 'cleaning', 'other', 'toxin-producing', 'neurotoxic', 'a', 'unknown',
            'genogroup', 'ii', 'i', 'toxin', 'or', 'poison', 'group', 'fish', 'plant', 'chemical', 'heavy',
            'fish', 'plant', 'agents']
toxins = [t for t in toxins if t not in stop_list]

In [175]:
toxins

['faecalis',
 'enterica',
 'vibrio',
 'giardia',
 'shellfish',
 'mycotoxins',
 'rotavirus',
 'coli',
 'enterocolitica',
 'boydii',
 'histamine',
 'flexneri',
 'spiralis',
 'bacterium',
 'parvum',
 'monocytogenes',
 'sonnei',
 'parasite',
 'adenovirus',
 'puffer',
 'brucella',
 'cloacae',
 'trichinella',
 'jejuni',
 'streptococcus',
 'campylobacter',
 'ciguatoxin',
 'cholerae',
 'cayatenensis',
 'virus',
 'enterococcus',
 'parahaemolyticus',
 'amnesic',
 'escherichia',
 'hepatitis',
 'hominis',
 'metals',
 'enteroaggregative',
 'shiga',
 'botulinum',
 'listeria',
 'salmonella',
 'coli,',
 'vulnificus',
 'enterotoxigenic',
 'shigella',
 'scombroid',
 'cereus',
 'cryptosporidium',
 'glutamate',
 'sapovirus',
 'norovirus',
 'cyclospora',
 'intestinalis',
 'toxins',
 'anisakis',
 'monosodium',
 'tetrodotoxin',
 'pesticides',
 'aureus',
 'paralytic',
 'enterobacter',
 'yersinia',
 'perfringens',
 'clostridium',
 'enteropathogenic',
 'staphylococcus',
 'bacillus']

In [168]:
foods = df['Food'].str.lower().str.replace(',', ' ').str.replace(';', ' ').str.replace('/', ' ').str.replace(':', ' ').str.replace('(', '').str.replace(')', '').str.replace('-', ' ').str.split(expand = True)

In [169]:
vocab = []
for row in range(foods.shape[0]):
    vocab.extend(foods.iloc[row,:].unique())

In [170]:
vocab = list(set([v for v in vocab if v is not None]))
stop_words = ['other', 'unspecified', '1%', 'contains', 'sa', 'con', 'cut', 'stir', 'royal',
             'head', 'heads', 'cooked', 'bites', 'unknown', 'chef', 'menu', 'foods', 'de', 'with',
             'easy', 'tri', 'w', 'multiple', 'bottled', 'food', 'see', 'of', 'made', 'sink',
             'based', '&', 'prepackaged', 'seven', 'soft', 'mini', 'ethnic', 'common', 'home', 'containing',
             'grilled', 'specialty', 'king', 'miscellaneous', 'sum', 'le', 'pre', 'packaged', 'whole',
             'mechanical', 'creamed', 'pasteurized', 'from', '2%', 'dietary', 'acid', 'commercial', 'pouch',
             'types', 'mashed', 'killed', 'cuisine', 'prime', 'dish', 'appetizer', 'waldorf', 'club', 'nine',
             "chef's", 'mixed', '*****', 'drink', 'tenderized', 'francaise', 'aus', 'acetic', 'crisp', 'tres',
             'puffed', 'ala', 'platter', 'twist', 'tray', 'shredded', 'lo', 'items', 'blackened', 'side', 'and',
             'white', 'homegrown', 'mediterranean', 'a', 'homemade', 'au', 'house', 'oriental', 'style',
             'ingredients', 'or', 'undercooked', 'free', 'served']
vocab = [v for v in vocab if not v in stop_words]

In [238]:
source = pd.DataFrame(columns = vocab, index = toxins)

In [239]:
source = source.fillna(0)

cannelloni    0
basil         0
pastry        0
filling       0
leafy         0
             ..
ranch         0
octopus       0
lettuce       0
peas          0
mango         0
Length: 877, dtype: int64

In [241]:
for row in tqdm(df.iterrows()):
    for food in vocab:
        for toxin in toxins:
            if food in row[1]['Food'] and toxin in row[1]['Species']:
                source.loc[toxin, food] +=1


0it [00:00, ?it/s]
1it [00:00,  3.10it/s]
2it [00:00,  3.13it/s]
3it [00:00,  3.17it/s]
4it [00:01,  3.21it/s]
5it [00:01,  3.25it/s]
6it [00:01,  3.28it/s]
7it [00:02,  3.29it/s]
8it [00:02,  3.29it/s]
9it [00:02,  3.29it/s]
10it [00:03,  3.28it/s]
11it [00:03,  3.23it/s]
12it [00:03,  3.19it/s]
13it [00:04,  3.15it/s]
14it [00:04,  3.12it/s]
15it [00:04,  3.11it/s]
16it [00:05,  3.09it/s]
17it [00:05,  3.11it/s]
18it [00:05,  3.18it/s]
19it [00:05,  3.21it/s]
20it [00:06,  3.23it/s]
21it [00:06,  3.25it/s]
22it [00:06,  3.27it/s]
23it [00:07,  3.27it/s]
24it [00:07,  3.25it/s]
25it [00:07,  3.25it/s]
26it [00:08,  3.25it/s]
27it [00:08,  3.21it/s]
28it [00:08,  3.21it/s]
29it [00:09,  3.16it/s]
30it [00:09,  3.17it/s]
31it [00:09,  3.10it/s]
32it [00:10,  3.02it/s]
33it [00:10,  3.01it/s]
34it [00:10,  3.07it/s]
35it [00:10,  3.12it/s]
36it [00:11,  3.15it/s]
37it [00:11,  3.15it/s]
38it [00:11,  3.15it/s]
39it [00:12,  3.14it/s]
40it [00:12,  3.12it/s]
41it [00:12,  3.10it/s]
42it 

296it [01:33,  3.10it/s]
297it [01:33,  3.10it/s]
298it [01:34,  3.11it/s]
299it [01:34,  3.11it/s]
300it [01:34,  3.12it/s]
301it [01:35,  3.07it/s]
302it [01:35,  3.10it/s]
303it [01:35,  3.11it/s]
304it [01:36,  3.08it/s]
305it [01:36,  3.10it/s]
306it [01:36,  3.13it/s]
307it [01:37,  3.12it/s]
308it [01:37,  3.12it/s]
309it [01:37,  3.09it/s]
310it [01:38,  3.13it/s]
311it [01:38,  3.17it/s]
312it [01:38,  3.21it/s]
313it [01:38,  3.23it/s]
314it [01:39,  3.24it/s]
315it [01:39,  3.23it/s]
316it [01:39,  3.17it/s]
317it [01:40,  3.18it/s]
318it [01:40,  3.22it/s]
319it [01:40,  3.23it/s]
320it [01:41,  3.26it/s]
321it [01:41,  3.27it/s]
322it [01:41,  3.28it/s]
323it [01:42,  3.26it/s]
324it [01:42,  3.26it/s]
325it [01:42,  3.26it/s]
326it [01:43,  3.26it/s]
327it [01:43,  3.25it/s]
328it [01:43,  3.23it/s]
329it [01:43,  3.22it/s]
330it [01:44,  3.21it/s]
331it [01:44,  3.21it/s]
332it [01:44,  3.22it/s]
333it [01:45,  3.23it/s]
334it [01:45,  3.24it/s]
335it [01:45,  3.25it/s]


588it [03:04,  3.27it/s]
589it [03:05,  3.25it/s]
590it [03:05,  3.24it/s]
591it [03:05,  3.25it/s]
592it [03:06,  3.28it/s]
593it [03:06,  3.31it/s]
594it [03:06,  3.33it/s]
595it [03:07,  3.34it/s]
596it [03:07,  3.34it/s]
597it [03:07,  3.34it/s]
598it [03:07,  3.34it/s]
599it [03:08,  3.36it/s]
600it [03:08,  3.37it/s]
601it [03:08,  3.38it/s]
602it [03:09,  3.37it/s]
603it [03:09,  3.35it/s]
604it [03:09,  3.35it/s]
605it [03:10,  3.35it/s]
606it [03:10,  3.35it/s]
607it [03:10,  3.33it/s]
608it [03:10,  3.33it/s]
609it [03:11,  3.32it/s]
610it [03:11,  3.33it/s]
611it [03:11,  3.31it/s]
612it [03:12,  3.33it/s]
613it [03:12,  3.34it/s]
614it [03:12,  3.34it/s]
615it [03:13,  3.34it/s]
616it [03:13,  3.36it/s]
617it [03:13,  3.36it/s]
618it [03:13,  3.36it/s]
619it [03:14,  3.35it/s]
620it [03:14,  3.33it/s]
621it [03:14,  3.34it/s]
622it [03:15,  3.35it/s]
623it [03:15,  3.37it/s]
624it [03:15,  3.35it/s]
625it [03:16,  3.33it/s]
626it [03:16,  3.32it/s]
627it [03:16,  3.31it/s]


880it [04:34,  3.43it/s]
881it [04:35,  3.44it/s]
882it [04:35,  3.45it/s]
883it [04:35,  3.46it/s]
884it [04:35,  3.45it/s]
885it [04:36,  3.43it/s]
886it [04:36,  3.44it/s]
887it [04:36,  3.44it/s]
888it [04:37,  3.45it/s]
889it [04:37,  3.44it/s]
890it [04:37,  3.43it/s]
891it [04:37,  3.45it/s]
892it [04:38,  3.45it/s]
893it [04:38,  3.43it/s]
894it [04:38,  3.43it/s]
895it [04:39,  3.41it/s]
896it [04:39,  3.39it/s]
897it [04:39,  3.38it/s]
898it [04:40,  3.38it/s]
899it [04:40,  3.39it/s]
900it [04:40,  3.39it/s]
901it [04:40,  3.38it/s]
902it [04:41,  3.38it/s]
903it [04:41,  3.38it/s]
904it [04:41,  3.38it/s]
905it [04:42,  3.37it/s]
906it [04:42,  3.37it/s]
907it [04:42,  3.38it/s]
908it [04:42,  3.38it/s]
909it [04:43,  3.39it/s]
910it [04:43,  3.38it/s]
911it [04:43,  3.36it/s]
912it [04:44,  3.38it/s]
913it [04:44,  3.38it/s]
914it [04:44,  3.36it/s]
915it [04:45,  3.36it/s]
916it [04:45,  3.35it/s]
917it [04:45,  3.35it/s]
918it [04:45,  3.36it/s]
919it [04:46,  3.35it/s]


1166it [05:59,  3.35it/s]
1167it [06:00,  3.37it/s]
1168it [06:00,  3.38it/s]
1169it [06:00,  3.37it/s]
1170it [06:01,  3.28it/s]
1171it [06:01,  3.21it/s]
1172it [06:01,  3.17it/s]
1173it [06:02,  3.14it/s]
1174it [06:02,  3.09it/s]
1175it [06:02,  3.16it/s]
1176it [06:03,  3.20it/s]
1177it [06:03,  3.24it/s]
1178it [06:03,  3.23it/s]
1179it [06:03,  3.23it/s]
1180it [06:04,  3.24it/s]
1181it [06:04,  3.23it/s]
1182it [06:04,  3.23it/s]
1183it [06:05,  3.22it/s]
1184it [06:05,  3.22it/s]
1185it [06:05,  3.19it/s]
1186it [06:06,  3.19it/s]
1187it [06:06,  3.18it/s]
1188it [06:06,  3.19it/s]
1189it [06:07,  3.21it/s]
1190it [06:07,  3.23it/s]
1191it [06:07,  3.24it/s]
1192it [06:07,  3.25it/s]
1193it [06:08,  3.24it/s]
1194it [06:08,  3.24it/s]
1195it [06:08,  3.24it/s]
1196it [06:09,  3.24it/s]
1197it [06:09,  3.25it/s]
1198it [06:09,  3.26it/s]
1199it [06:10,  3.26it/s]
1200it [06:10,  3.25it/s]
1201it [06:10,  3.26it/s]
1202it [06:11,  3.31it/s]
1203it [06:11,  3.34it/s]
1204it [06:1

...

5960it [30:13,  3.19it/s]
5961it [30:14,  3.20it/s]
5962it [30:14,  3.22it/s]
5963it [30:14,  3.26it/s]
5964it [30:15,  3.28it/s]
5965it [30:15,  3.27it/s]
5966it [30:15,  3.25it/s]
5967it [30:15,  3.26it/s]
5968it [30:16,  3.28it/s]
5969it [30:16,  3.29it/s]
5970it [30:16,  3.29it/s]
5971it [30:17,  3.31it/s]
5972it [30:17,  3.30it/s]
5973it [30:17,  3.31it/s]
5974it [30:18,  3.32it/s]
5975it [30:18,  3.32it/s]
5976it [30:18,  3.33it/s]
5977it [30:18,  3.33it/s]
5978it [30:19,  3.24it/s]
5979it [30:19,  3.23it/s]
5980it [30:19,  3.25it/s]
5981it [30:20,  3.28it/s]
5982it [30:20,  3.28it/s]
5983it [30:20,  3.26it/s]
5984it [30:21,  3.28it/s]
5985it [30:21,  3.29it/s]
5986it [30:21,  3.32it/s]
5987it [30:22,  3.30it/s]
5988it [30:22,  3.26it/s]
5989it [30:22,  3.28it/s]
5990it [30:22,  3.28it/s]
5991it [30:23,  3.28it/s]
5992it [30:23,  3.28it/s]
5993it [30:23,  3.28it/s]
5994it [30:24,  3.28it/s]
5995it [30:24,  3.24it/s]
5996it [30:24,  3.22it/s]
5997it [30:25,  3.23it/s]
5998it [30:2

6242it [31:39,  3.19it/s]
6243it [31:40,  3.20it/s]
6244it [31:40,  3.16it/s]
6245it [31:40,  3.17it/s]
6246it [31:41,  3.19it/s]
6247it [31:41,  3.19it/s]
6248it [31:41,  3.21it/s]
6249it [31:42,  3.22it/s]
6250it [31:42,  3.25it/s]
6251it [31:42,  3.26it/s]
6252it [31:42,  3.29it/s]
6253it [31:43,  3.30it/s]
6254it [31:43,  3.30it/s]
6255it [31:43,  3.31it/s]
6256it [31:44,  3.31it/s]
6257it [31:44,  3.33it/s]
6258it [31:44,  3.33it/s]
6259it [31:45,  3.32it/s]
6260it [31:45,  3.33it/s]
6261it [31:45,  3.32it/s]
6262it [31:45,  3.31it/s]
6263it [31:46,  3.30it/s]
6264it [31:46,  3.31it/s]
6265it [31:46,  3.32it/s]
6266it [31:47,  3.30it/s]
6267it [31:47,  3.30it/s]
6268it [31:47,  3.30it/s]
6269it [31:48,  3.31it/s]
6270it [31:48,  3.32it/s]
6271it [31:48,  3.33it/s]
6272it [31:48,  3.32it/s]
6273it [31:49,  3.32it/s]
6274it [31:49,  3.31it/s]
6275it [31:49,  3.31it/s]
6276it [31:50,  3.31it/s]
6277it [31:50,  3.31it/s]
6278it [31:50,  3.32it/s]
6279it [31:51,  3.30it/s]
6280it [31:5

6524it [33:05,  3.37it/s]
6525it [33:05,  3.37it/s]
6526it [33:06,  3.36it/s]
6527it [33:06,  3.37it/s]
6528it [33:06,  3.39it/s]
6529it [33:07,  3.39it/s]
6530it [33:07,  3.38it/s]
6531it [33:07,  3.38it/s]
6532it [33:07,  3.39it/s]
6533it [33:08,  3.38it/s]
6534it [33:08,  3.34it/s]
6535it [33:08,  3.35it/s]
6536it [33:09,  3.34it/s]
6537it [33:09,  3.35it/s]
6538it [33:09,  3.34it/s]
6539it [33:10,  3.33it/s]
6540it [33:10,  3.33it/s]
6541it [33:10,  3.33it/s]
6542it [33:10,  3.34it/s]
6543it [33:11,  3.33it/s]
6544it [33:11,  3.33it/s]
6545it [33:11,  3.33it/s]
6546it [33:12,  3.30it/s]
6547it [33:12,  3.29it/s]
6548it [33:12,  3.29it/s]
6549it [33:13,  3.29it/s]
6550it [33:13,  3.29it/s]
6551it [33:13,  3.32it/s]
6552it [33:13,  3.33it/s]
6553it [33:14,  3.34it/s]
6554it [33:14,  3.34it/s]
6555it [33:14,  3.34it/s]
6556it [33:15,  3.36it/s]
6557it [33:15,  3.36it/s]
6558it [33:15,  3.38it/s]
6559it [33:16,  3.39it/s]
6560it [33:16,  3.37it/s]
6561it [33:16,  3.38it/s]
6562it [33:1

6806it [34:29,  3.37it/s]
6807it [34:29,  3.40it/s]
6808it [34:29,  3.42it/s]
6809it [34:30,  3.36it/s]
6810it [34:30,  3.38it/s]
6811it [34:30,  3.36it/s]
6812it [34:30,  3.38it/s]
6813it [34:31,  3.39it/s]
6814it [34:31,  3.40it/s]
6815it [34:31,  3.40it/s]
6816it [34:32,  3.40it/s]
6817it [34:32,  3.40it/s]
6818it [34:32,  3.40it/s]
6819it [34:32,  3.40it/s]
6820it [34:33,  3.41it/s]
6821it [34:33,  3.42it/s]
6822it [34:33,  3.41it/s]
6823it [34:34,  3.42it/s]
6824it [34:34,  3.41it/s]
6825it [34:34,  3.41it/s]
6826it [34:35,  3.40it/s]
6827it [34:35,  3.40it/s]
6828it [34:35,  3.38it/s]
6829it [34:35,  3.39it/s]
6830it [34:36,  3.38it/s]
6831it [34:36,  3.39it/s]
6832it [34:36,  3.39it/s]
6833it [34:37,  3.39it/s]
6834it [34:37,  3.39it/s]
6835it [34:37,  3.38it/s]
6836it [34:37,  3.38it/s]
6837it [34:38,  3.38it/s]
6838it [34:38,  3.38it/s]
6839it [34:38,  3.37it/s]
6840it [34:39,  3.35it/s]
6841it [34:39,  3.33it/s]
6842it [34:39,  3.32it/s]
6843it [34:40,  3.35it/s]
6844it [34:4

7088it [35:54,  3.28it/s]
7089it [35:54,  3.30it/s]
7090it [35:54,  3.31it/s]
7091it [35:55,  3.33it/s]
7092it [35:55,  3.34it/s]
7093it [35:55,  3.34it/s]
7094it [35:56,  3.34it/s]
7095it [35:56,  3.36it/s]
7096it [35:56,  3.36it/s]
7097it [35:56,  3.38it/s]
7098it [35:57,  3.39it/s]
7099it [35:57,  3.37it/s]
7100it [35:57,  3.38it/s]
7101it [35:58,  3.33it/s]
7102it [35:58,  3.32it/s]
7103it [35:58,  3.33it/s]
7104it [35:59,  3.33it/s]
7105it [35:59,  3.31it/s]
7106it [35:59,  3.31it/s]
7107it [35:59,  3.34it/s]
7108it [36:00,  3.36it/s]
7109it [36:00,  3.37it/s]
7110it [36:00,  3.36it/s]
7111it [36:01,  3.36it/s]
7112it [36:01,  3.36it/s]
7113it [36:01,  3.36it/s]
7114it [36:01,  3.37it/s]
7115it [36:02,  3.38it/s]
7116it [36:02,  3.38it/s]
7117it [36:02,  3.40it/s]
7118it [36:03,  3.40it/s]
7119it [36:03,  3.41it/s]
7120it [36:03,  3.40it/s]
7121it [36:04,  3.39it/s]
7122it [36:04,  3.37it/s]
7123it [36:04,  3.35it/s]
7124it [36:04,  3.35it/s]
7125it [36:05,  3.34it/s]
7126it [36:0

In [246]:
source.sum()

cannelloni      4
basil          12
pastry         21
filling         6
leafy          16
             ... 
ranch          37
octopus         6
lettuce       574
peas           20
mango          14
Length: 877, dtype: int64